#### Relevant Imports

In [44]:
from sqlalchemy import create_engine, text
import json
from collections import defaultdict
from typing import List, Dict, Any
import csv
from groq import Groq
import dotenv
import openai

**SQLite DB**  
Let us connect to the SQLite Sample Database what we have  


In [2]:
# There is an engine instance created, which can handle multiple connetions
sql_engine = create_engine("sqlite:///Sample_1 - Copy.db")
conn_1 = sql_engine.connect ()

**Output in JSON**  
From a specific search Criteria, SQL query is executed to get the results  
Further this is used as context to LLM to answer a question

In [ ]:
# Get the details of Service Rep for customers in a country 
Country = 'USA'
result = conn_1.execute (text(f"""
                                SELECT
                                    c.FirstName || ' ' || c.LastName AS CustomerName,
                                    c.State AS CustomerState,
                                    c.Country AS CustomerCountry,
                                    e.FirstName || ' ' || e.LastName AS SupportRepName,
                                    e.State AS RepState,
                                    e.Country AS RepCountry,
                                    e.Email AS SupportRepEmail
                                FROM Customer c
                                LEFT JOIN Employee e ON c.SupportRepId = e.EmployeeId
                                WHERE c.Country = '{Country}';  
                              """))

# Query output into JSON
rows = [row._asdict () for row in result]

# Convert to JSON
json_output = json.dumps(rows,  indent=2)
print (json_output)


**Use it in LLM**
Define a Groq Client and invoke LLM to answer question  
use the fetched information as context for query

In [59]:
# Load Env variable and define a Groq Client
dotenv.load_dotenv ()
client = Groq()
model = "llama3-70b-8192"

# client = openai.OpenAI ()
# model = "gpt-4o-mini"

> Invoke of Groq API for llama model  
> Specific instructions are required

In [ ]:
Instr = "Using the context given, provide response to the user question or statement.\
        Context is provided as JSON information.\
        Answer to the question precisely"

Question = f"How many services reps the {Country} customers have?"
# Question = f"Who can the {Country} customers contact if they have an issue?"
# Question = f"Are the service reps in same place of the customers?"
# Question = f"Tell me how much time diff b/w victor and his service guy"
# Question = f"Who handles more customers?"

messages=[
    {
        "role": "system",
        "content": Instr
    },

    {
        "role": "user",
        "content":"Context : \n"+ json_output + "Query : \n" + Question
    }
]
completion = client.chat.completions.create(
    messages=messages,
    model="llama3-70b-8192",
    # temperature=0.0
)

print (completion.choices[0].message.content)

> Additional details for manager is also fetched  
> Only contact details are gathered 

In [ ]:
# Get the details of Service Rep for customers in a country including the manager details
Country = 'Canada'
result = conn_1.execute (text(f"""
                                SELECT
                                    c.FirstName || ' ' || c.LastName AS CustomerName,
                                    c.State AS CustomerState,
                                    e.FirstName || ' ' || e.LastName AS SupportRepName,
                                    e.Email AS SupportRepEmail,
                                    m.FirstName || ' ' || m.LastName AS ManagerName,
                                    m.Email AS ManagerEmail
                                FROM Customer c
                                LEFT JOIN Employee e ON c.SupportRepId = e.EmployeeId
                                LEFT JOIN Employee m ON e.ReportsTo = m.EmployeeId
                                WHERE c.Country = '{Country}';
                              """))

# Make each row into a sentence
rows = result.fetchall()
context = []
for row in rows:
    Customer, C_State, Rep, Rep_Mail, Manager, Mgr_Mail = row
    sentence = f"{Customer} from state {C_State} may contact sales rep {Rep} @ {Rep_Mail}, who reports to the manager {Manager} @ {Mgr_Mail}"
    context.append(sentence)

for line in context: 
  print (line)

In [ ]:
Instr = "Using the context given, provide response to the user question or statement.\
        Answer to the question precisely"

Question = f"Tell me how is the escalation matrix organised"
# Question = f"Who can the {Country} customers contact if the service support is not satisfactory"
# Question = f"How many Reps report to the managers?"

messages=[
    {
        "role": "system",
        "content": Instr
    },

    {
        "role": "user",
        "content":"Context : \n"+ "".join(context) + "\nQuery : \n" + Question
    }
]
completion = client.chat.completions.create(
    messages=messages,
    model="llama3-70b-8192",
    # temperature=0.0
)

print (completion.choices[0].message.content)

**More Data**  
Data fetched for customers invoice amount and date for each Sales Rep  
Further this is captured in JSON

In [33]:
def group_json_by_n_columns(data: List[Dict[str, Any]], n: int) -> Dict:
    
    if not data:
        return {}

    # If n is 1, there is no need for re-aligning
    if n < 1:
        raise ValueError("n must be at least 1")

    # Check if those many columns are there
    column_keys = list(data[0].keys())
    if n > len(column_keys):
        raise ValueError(f"n = {n} exceeds number of columns = {len(column_keys)}")

    group_keys = column_keys[:n]

    def nest(rows: List[Dict[str, Any]], keys: List[str]) -> Dict:
        if not keys:
            return rows

        result = defaultdict(list)
        current_key = keys[0]

        for row in rows:
            group_value = row[current_key]
            remaining_row = {k: v for k, v in row.items() if k != current_key}
            result[group_value].append(remaining_row)

        # Recurse for each group
        return {
            k: nest(v, keys[1:]) for k, v in result.items()
        }

    return nest(data, group_keys)

In [ ]:
# For Each Service Rep from Canada, their customer details, total sales per customer and the last purchase date 
Country = 'Canada'
result = conn_1.execute (text(f"""
                                SELECT
                                    E.FirstName || ' ' || E.LastName AS SupportRep,
                                    E.Email AS SupportEmail,
                                    E.Country AS Rep_Country,
                                    C.FirstName || ' ' || C.LastName AS Customer,
                                    C.Email AS CustomerEmail,
                                    SUM(I.Total) AS Total_Invoice,
                                    MAX(I.InvoiceDate) AS Last_Invoice
                                FROM
                                    Customer AS C
                                JOIN
                                    Invoice AS I ON C.CustomerId = I.CustomerId
                                JOIN
                                    Employee AS E ON C.SupportRepId = E.EmployeeId
                                WHERE
                                    E.Country = 'Canada'
                                GROUP BY    
                                    C.CustomerId
                                ORDER BY
                                    SupportRep, Customer;
                              """))

# Query output into JSON
rows = [row._asdict () for row in result]

# Re-align
result = group_json_by_n_columns(rows, 1)

# Convert to JSON
json_output = json.dumps(result,  indent=2)
print (json_output)

with open ('Output.json', 'w') as f:
    f.write (json_output)

In [ ]:
Instr = "Using the context given, provide response to the user question or statement.\
        Context is in JSON format.\
        Respond in bullet point"

Question = f"When was the last purchase made by Jane's customer?"
# Question = f"Puja realised the invoice shall be 20.0. How much is the dispute amount? whom to contact?"
# Question = f"Which service rep has cumulative customer invoice the highest?"

messages=[
    {
        "role": "system",
        "content": Instr
    },

    {
        "role": "user",
        "content":"Context : \n"+ json_output + "Query : \n" + Question
    }
]
completion = client.chat.completions.create(
    messages=messages,
    model=model,
    # temperature=0.0
)

print (completion.choices[0].message.content)

>Numerical calculations across various parts of data, not to rely on LLM  
>Instead forumate the SQL to do the math part and use LLM to comprehend and respond

In [ ]:
# For Service Rep in Canada, find the total invoice cumulative invoice for all their customers
result = conn_1.execute (text(f"""
                                SELECT
                                    E.FirstName || ' ' || E.LastName AS SupportRep,
                                    E.Email AS SupportEmail,
                                    E.Country AS Rep_Country,
                                    SUM(I.Total) AS Total_Invoice,
                                    MAX(I.InvoiceDate) AS Last_Invoice
                                FROM
                                    Customer AS C
                                JOIN
                                    Invoice AS I ON C.CustomerId = I.CustomerId
                                JOIN
                                    Employee AS E ON C.SupportRepId = E.EmployeeId
                                WHERE
                                    E.Country = 'Canada'
                                GROUP BY
                                    E.EmployeeId
                              """))

# Query output into JSON
rows = [row._asdict () for row in result]

# Re-align
result = group_json_by_n_columns(rows, 1)

# Convert to JSON
json_output = json.dumps(result,  indent=2)
print (json_output)

with open ('Output_summary.json', 'w') as f:
    f.write (json_output)

In [ ]:
Instr = "Using the context given, provide response to the user question or statement.\
        Context is in JSON format.\
        Respond in bullet point"

Question = f"Which service rep has cumulative customer invoice the highest?"

messages=[
    {
        "role": "system",
        "content": Instr
    },

    {
        "role": "user",
        "content":"Context : \n"+ json_output + "Query : \n" + Question
    }
]
completion = client.chat.completions.create(
    messages=messages,
    model=model,
    # temperature=0.0
)

print (completion.choices[0].message.content)